# Краткое сравнение моделей
| Модель                     |       TEST accuracy |       TEST f1_macro | refit / train time (s) |
| -------------------------- | ------------------: | ------------------: | ---------------------: |
| XGBClassifier              |    99.68% (0.99675) |   99.67% (0.996732) |                   2.47 |
| RandomForestClassifier     |      99.60% (0.996) |    99.60% (0.99599) |                   2.14 |
| LogisticRegression (C=0.1) | **99.85% (0.9985)** | **99.85% (0.9985)** |               **0.09** |


Дополнительные наблюдения:

* Все три модели дают почти идеальные метрики на тесте.

* LogisticRegression показывает наилучшие численные метрики и самое короткое время обучения.

* XGB и RF близки по качеству, немного медленнее; XGB даёт чуть лучшее качество, чем RF.

# Для дальнейшей работы и улучшений выбираю Logistic Regression

### Обоснование:

* Лучшие метрики на тесте (accuracy и f1_macro ≈ 99.85%).

* Простота и интерпретируемость — коэффициенты линейной модели легко объяснить и логировать.

* Меньший размер и низкая латентность — быстрее inference, проще деплой (меньше памяти/CPU).

* Лучше калибровка вероятностей по умолчанию 


Разница в качестве между Logistic и более сложными моделями невелика или отсутствует, поэтому дополнительная сложность XGB/RF не оправдана.

# `SHAP отчет для Logistic Regresion`


### **Модель**: LogisticRegression обученная на 12 признаках.
### **SHAP-подход** Объясняю через вероятности .SHAP‑значения интерпретируются как сдвиг вероятности класса относительно базовой вероятности

### **Список признаков** : 'social_energy', 'alone_time_preference', 'talkativeness', 'deep_reflection', 'group_comfort', 'party_liking', 'leadership', 'risk_taking', 'public_speaking_comfort', 'excitement_seeking', 'adventurousness', 'reading_habit'


### `Глобальная важность признаков` по классам


![bar](../../reports/figures/SHAP_bar_Ambivert.png)

![bar](../../reports/figures/SHAP_bar_Introvert.png)

![bar](../../reports/figures/SHAP_bar_Extrovert.png)

### На графиках видно что модель на всех трех классах опираеться в основном на оди и тот же блок признаков . Признаки описываюшие социальную активность. 
### **Топ драйвер** Party Liking имеет самый большой вклад по модулю для всех классов.

### Модель строит шкалу «экстраверсия—интроверсия». Ambivert представлен как промежуточный регион на этой шкале, поэтому те же признаки, что определяют крайние классы, оказываются топ-важными и для Ambivert.


![bar](../../reports/figures/SHAP_beeswarm_Extrovert.png)
![bar](../../reports/figures/SHAP_beeswarm_Introvert.png)
![bar](../../reports/figures/SHAP_beeswarm_Ambivert.png)

### По beeswarm видно стабильные закономерности
### Для класса `EXTROVERT`:
* ### Большие значения в Признаках party_liking, talkativeness, excitement_seeking **повышают** вероятность класса Extrovert.
* ###  Большие значения  alone_time_preference ,reading_habit и deep_reflection  **понижают** вероятность. Уводят в минус как "Интровертные маркеры"

### Для класса `Introvet` наблюдаем почти зеркальную картину :
* ###  Большие значения alone_time_preference ,reading_habit и deep_reflection **повышают** вероятность пренодлежности к классу
* ### Большие значения party_liking, talkativeness, social_energy **понижают** вероятность

### Класс `Ambivert`
### Вклад многих признаков двунаправленый. То есть часто класс Ambivert принимаеться когда признаки не экстримальные и часть признаков тянет в сторону экстроверсии а часть в сторону Интроверсии.
### Можно сказать что класс Ambivert определяется сочетанием умеренной социальной активности и наличия как интровертных, так и экстравертных черт

# Посмотрим на локальных примерах

### Waterfall график  предсказан **Introvert**


![bar](../../reports/figures/INTROVET.png)

### Waterfall график  предсказан **Extrovert**
![bar](../../reports/figures/EXTROVERT.png)

### Waterfall график  предсказан **Ambivert**
![bar](../../reports/figures/AMBIVERT.png)

### Заключение 
### Основные факторы модели: наибольшее влияние на классификацию оказывают party_liking, talkativeness, alone_time_preference, excitement_seeking, public_speaking_comfort, social_energy. Эти признаки формируют доминирующую ось “социальная активность ↔ предпочтение одиночества”.

* ### Extrovert: высокая вероятность класса связана с высокими значениями социальных признаков (любовь к вечеринкам, разговорчивость, социальная энергия, комфорт публичных выступлений, поиск возбуждения).
* ### Introvert: высокая вероятность класса связана с предпочтением одиночества и “рефлексивно‑индивидуальными” паттернами (чтение, глубокая рефлексия), а также с низкими значениями социальных признаков.
* ### Ambivert: класс определяется смешанным профилем — модель относит к Ambivert объекты, где признаки не дают сильного перекоса ни в Extrovert, ни в Introvert; наблюдается “баланс” между социальными и индивидуальными чертами.
